In [1]:
#-RAKI imports
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io as sio
import numpy as np
import numpy.matlib
import time
import os

#-My imports
from utils import signalprocessing as sig

Instructions for updating:
non-resource variables are not supported in the long term


# RAKI definitions 

In [2]:
def weight_variable(shape,vari_name):                   
    initial = tf.truncated_normal(shape, stddev=0.1,dtype=tf.float32)
    return tf.Variable(initial,name = vari_name)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape,dtype=tf.float32)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def conv2d_dilate(x, W,dilate_rate):
    return tf.nn.convolution(x, W,padding='VALID',dilation_rate = [1,dilate_rate])

#### LEANING FUNCTION ####
def learning(ACS_input,target_input,accrate_input,sess):
    
    input_ACS = tf.placeholder(tf.float32, [1, ACS_dim_X,ACS_dim_Y,ACS_dim_Z])                                  
    input_Target = tf.placeholder(tf.float32, [1, target_dim_X,target_dim_Y,target_dim_Z])         
    
    Input = tf.reshape(input_ACS, [1, ACS_dim_X, ACS_dim_Y, ACS_dim_Z])         

    [target_dim0,target_dim1,target_dim2,target_dim3] = np.shape(target)

    W_conv1 = weight_variable([kernel_x_1, kernel_y_1, ACS_dim_Z, layer1_channels],'W1') 
    h_conv1 = tf.nn.relu(conv2d_dilate(Input, W_conv1,accrate_input)) 

    W_conv2 = weight_variable([kernel_x_2, kernel_y_2, layer1_channels, layer2_channels],'W2')
    h_conv2 = tf.nn.relu(conv2d_dilate(h_conv1, W_conv2,accrate_input))

    W_conv3 = weight_variable([kernel_last_x, kernel_last_y, layer2_channels, target_dim3],'W3')
    h_conv3 = conv2d_dilate(h_conv2, W_conv3,accrate_input)

    error_norm = tf.norm(input_Target - h_conv3)       
    train_step = tf.train.AdamOptimizer(LearningRate).minimize(error_norm)

    if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
        init = tf.initialize_all_variables()
    else:
        init = tf.global_variables_initializer()
    sess.run(init)

    error_prev = 1 
    for i in range(MaxIteration):
        
        sess.run(train_step, feed_dict={input_ACS: ACS, input_Target: target})
        if i % 100 == 0:                                                                      
            error_now=sess.run(error_norm,feed_dict={input_ACS: ACS, input_Target: target})    
            print('The',i,'th iteration gives an error',error_now)                             
            
            
        
    error = sess.run(error_norm,feed_dict={input_ACS: ACS, input_Target: target})
    return [sess.run(W_conv1),sess.run(W_conv2),sess.run(W_conv3),error]  


def cnn_3layer(input_kspace,w1,b1,w2,b2,w3,b3,acc_rate,sess):                
    h_conv1 = tf.nn.relu(conv2d_dilate(input_kspace, w1,acc_rate)) 
    h_conv2 = tf.nn.relu(conv2d_dilate(h_conv1, w2,acc_rate))
    h_conv3 = conv2d_dilate(h_conv2, w3,acc_rate) 
    return sess.run(h_conv3)           

# Loading k-space and defining some operators for raki purposes

In [3]:
fft2c_raki  = lambda x: sig.fft(sig.fft(x,0),1)
ifft2c_raki = lambda x: sig.ifft(sig.ifft(x,0),1)

In [4]:
image_coils_truth  = sio.loadmat('data/img_grappa_32chan.mat')['IMG']

kspace_truth_raki  = fft2c_raki(image_coils_truth)
[M,N,C] = kspace_truth_raki.shape

# Setting RAKI network parameters 

In [5]:
#Acquisition/Acceleration Values
Rx     = 1
all_Ry = [5,6]

acsx     = M
all_acsy = [20,24,30,36,40]

#### Network Parameters ####
all_kernel_x_1 = [5]
all_kernel_y_1 = [2]

all_kernel_x_2 = [1]
all_kernel_y_2 = [1]

all_kernel_last_x = [3]
all_kernel_last_y = [2]

all_layer1_channels = [32] 
all_layer2_channels = [8]

MaxIteration = 1000     #Default = 1000
LearningRate = 3e-3  #Default = 3e-3

#kernel_x_1 = 3       #Default = 5
#kernel_y_1 = 2       #Default = 2

#kernel_x_2 = 1       #Default = 1
#kernel_y_2 = 1       #Default = 1

#kernel_last_x = 3    #Default = 3
#kernel_last_y = 2    #Default = 2

#layer1_channels = 32 #Default = 32
#layer2_channels = 8  #Default = 8



# Trying to copy the RAKI implementation here 

## preparing for ablation loop 

In [6]:
all_kspace_recons = np.zeros((M,N,C,len(all_kernel_x_1),len(all_kernel_y_1),len(all_kernel_x_2),\
                          len(all_kernel_y_2),len(all_kernel_last_x),len(all_kernel_last_y),\
                          len(all_layer1_channels),len(all_layer2_channels),len(all_Ry),len(all_acsy)),\
                             dtype = complex)

In [ ]:
for aa in range(len(all_kernel_x_1)):
    for bb in range(len(all_kernel_y_1)):
        for cc in range(len(all_kernel_x_2)):
            for dd in range(len(all_kernel_y_2)):
                for ee in range(len(all_kernel_last_x)):
                    for ff in range(len(all_kernel_last_y)):
                        for gg in range(len(all_layer1_channels)):
                            for hh in range(len(all_layer2_channels)):
                                for ii in range(len(all_Ry)):
                                    for jj in range(len(all_acsy)):
                                        kernel_x_1 = all_kernel_x_1[aa]       #Default = 5
                                        kernel_y_1 = all_kernel_y_1[bb]       #Default = 2

                                        kernel_x_2 = all_kernel_x_2[cc]       #Default = 1
                                        kernel_y_2 = all_kernel_y_2[dd]       #Default = 1

                                        kernel_last_x = all_kernel_last_x[ee]    #Default = 3
                                        kernel_last_y = all_kernel_last_y[ff]    #Default = 2

                                        layer1_channels = all_layer1_channels[gg] #Default = 32
                                        layer2_channels = all_layer2_channels[hh]  #Default = 8

                                        Ry   = all_Ry[ii]
                                        acsy = all_acsy[jj]
                                        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
                                        print("R: %d || acs: %d || x1: %d || y1: %d || x2: %d || y2: %d || x3: %d || y3: %d || l1ch: %d || l2ch: %d" \
                                              % (Ry,acsy,kernel_x_1,kernel_y_1,kernel_x_2,kernel_last_x,kernel_last_x,kernel_last_y,layer1_channels,layer2_channels))
                                        print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

                                        #Generate zero-filled ACS
                                        acsregionX = np.arange(M//2 - acsx // 2,M//2 + acsx//2) 
                                        acsregionY = np.arange(N//2 - acsy // 2,N//2 + acsy//2) 

                                        kspace_raki_undersampled_withacs = np.zeros((M,N,C),dtype = complex)
                                        kspace_raki_undersampled_withacs[::Rx,::Ry,:] = kspace_truth_raki[::Rx,::Ry,:]
                                        kspace_raki_undersampled_withacs[acsregionX[0]:acsregionX[acsx-1]+1,acsregionY[0]:acsregionY[acsy-1]+1,:]\
                                            = kspace_truth_raki[acsregionX[0]:acsregionX[acsx-1]+1,acsregionY[0]:acsregionY[acsy-1]+1,:]

                                        kspace = np.copy(kspace_raki_undersampled_withacs)
                                        no_ACS_flag = 0;
                                        normalize = 0.015/np.max(abs(kspace[:]))
                                        kspace = np.multiply(kspace,normalize) 

                                        [m1,n1,no_ch] = np.shape(kspace)
                                        no_inds = 1

                                        kspace_all = np.copy(kspace);
                                        kx = np.transpose(np.int32([(range(1,m1+1))]))                  
                                        ky = np.int32([(range(1,n1+1))])

                                        kspace = np.copy(kspace_all)
                                        mask = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),0),1))>0; 
                                        picks = np.where(mask == 1);                                  
                                        kspace = kspace[:,np.int32(picks[0][0]):n1+1,:]
                                        kspace_all = kspace_all[:,np.int32(picks[0][0]):n1+1,:]  

                                        kspace_NEVER_TOUCH = np.copy(kspace_all)

                                        mask = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),0),1))>0;  
                                        picks = np.where(mask == 1);                                  
                                        d_picks = np.diff(picks,1)  
                                        indic = np.where(d_picks == 1);

                                        mask_x = np.squeeze(np.matlib.sum(np.matlib.sum(np.abs(kspace),2),1))>0;
                                        picks_x = np.where(mask_x == 1);
                                        x_start = picks_x[0][0]
                                        x_end = picks_x[0][-1]

                                        if np.size(indic)==0:    
                                            no_ACS_flag=1;       
                                            print('No ACS signal in input data, using individual ACS file.')
                                            matfn = 'ACS.mat'   
                                            ACS = sio.loadmat(matfn)
                                            ACS = ACS['ACS']     
                                            [ACS_dim_X, ACS_dim_Y, ACS_dim_Z] = np.shape(ACS)
                                            ACS_re = np.zeros([ACS_dim_X,ACS_dim_Y,ACS_dim_Z*2])
                                            ACS_re[:,:,0:no_ch] = np.real(ACS)
                                            ACS_re[:,:,no_ch:no_ch*2] = np.imag(ACS)
                                        else:
                                            no_ACS_flag=0;
                                            print('ACS signal found in the input data')
                                            indic = indic[1][:]
                                            center_start = picks[0][indic[0]];
                                            center_end = picks[0][indic[-1]+1];
                                            ACS = kspace[x_start:x_end+1,center_start:center_end+1,:]
                                            [ACS_dim_X, ACS_dim_Y, ACS_dim_Z] = np.shape(ACS)
                                            ACS_re = np.zeros([ACS_dim_X,ACS_dim_Y,ACS_dim_Z*2])
                                            ACS_re[:,:,0:no_ch] = np.real(ACS)
                                            ACS_re[:,:,no_ch:no_ch*2] = np.imag(ACS)

                                        acc_rate = d_picks[0][0]
                                        no_channels = ACS_dim_Z*2

                                        w1_all = np.zeros([kernel_x_1, kernel_y_1, no_channels, layer1_channels, no_channels],dtype=np.float32)
                                        w2_all = np.zeros([kernel_x_2, kernel_y_2, layer1_channels,layer2_channels,no_channels],dtype=np.float32)
                                        w3_all = np.zeros([kernel_last_x, kernel_last_y, layer2_channels,acc_rate - 1, no_channels],dtype=np.float32)    

                                        b1_flag = 0;
                                        b2_flag = 0;                       
                                        b3_flag = 0;

                                        if (b1_flag == 1):
                                            b1_all = np.zeros([1,1, layer1_channels,no_channels]);
                                        else:
                                            b1 = []

                                        if (b2_flag == 1):
                                            b2_all = np.zeros([1,1, layer2_channels,no_channels])
                                        else:
                                            b2 = []

                                        if (b3_flag == 1):
                                            b3_all = np.zeros([1,1, layer3_channels, no_channels])
                                        else:
                                            b3 = []

                                        target_x_start = np.int32(np.ceil(kernel_x_1/2) + np.floor(kernel_x_2/2) + np.floor(kernel_last_x/2) -1); 
                                        target_x_end = np.int32(ACS_dim_X - target_x_start -1); 

                                        time_ALL_start = time.time()

                                        [ACS_dim_X, ACS_dim_Y, ACS_dim_Z] = np.shape(ACS_re)
                                        ACS = np.reshape(ACS_re, [1,ACS_dim_X, ACS_dim_Y, ACS_dim_Z]) 
                                        ACS = np.float32(ACS)  

                                        target_y_start = np.int32((np.ceil(kernel_y_1/2)-1) + (np.ceil(kernel_y_2/2)-1) + (np.ceil(kernel_last_y/2)-1)) * acc_rate;     
                                        target_y_end = ACS_dim_Y  - np.int32((np.floor(kernel_y_1/2) + np.floor(kernel_y_2/2) + np.floor(kernel_last_y/2))) * acc_rate -1;

                                        target_dim_X = target_x_end - target_x_start + 1
                                        target_dim_Y = target_y_end - target_y_start + 1
                                        target_dim_Z = acc_rate - 1

                                        print('go!')
                                        time_Learn_start = time.time() 

                                        errorSum = 0;
                                        config = tf.ConfigProto()


                                        for ind_c in range(ACS_dim_Z):

                                            sess = tf.Session(config=config)
                                            # set target lines
                                            target = np.zeros([1,target_dim_X,target_dim_Y,target_dim_Z])
                                            print('learning channel #',ind_c+1)
                                            time_channel_start = time.time()

                                            for ind_acc in range(acc_rate-1):
                                                target_y_start = np.int32((np.ceil(kernel_y_1/2)-1) + (np.ceil(kernel_y_2/2)-1) + (np.ceil(kernel_last_y/2)-1)) * acc_rate + ind_acc + 1 
                                                target_y_end = ACS_dim_Y  - np.int32((np.floor(kernel_y_1/2) + (np.floor(kernel_y_2/2)) + np.floor(kernel_last_y/2))) * acc_rate + ind_acc
                                                target[0,:,:,ind_acc] = ACS[0,target_x_start:target_x_end + 1, target_y_start:target_y_end +1,ind_c];

                                            # learning

                                            [w1,w2,w3,error]=learning(ACS,target,acc_rate,sess) 
                                            w1_all[:,:,:,:,ind_c] = w1
                                            w2_all[:,:,:,:,ind_c] = w2
                                            w3_all[:,:,:,:,ind_c] = w3                               
                                            time_channel_end = time.time()
                                            print('Time Cost:',time_channel_end-time_channel_start,'s')
                                            print('Norm of Error = ',error)
                                            errorSum = errorSum + error

                                            sess.close()
                                            tf.reset_default_graph()

                                        time_Learn_end = time.time();

                                        print('lerning step costs:',(time_Learn_end - time_Learn_start)/60,'min')
                                        kspace_recon_all = np.copy(kspace_all)
                                        kspace_recon_all_nocenter = np.copy(kspace_all)

                                        kspace = np.copy(kspace_all)

                                        over_samp = np.setdiff1d(picks,np.int32([range(0, n1,acc_rate)]))
                                        kspace_und = kspace
                                        kspace_und[:,over_samp,:] = 0;
                                        [dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z] = np.shape(kspace_und)

                                        kspace_und_re = np.zeros([dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z*2])
                                        kspace_und_re[:,:,0:dim_kspaceUnd_Z] = np.real(kspace_und)
                                        kspace_und_re[:,:,dim_kspaceUnd_Z:dim_kspaceUnd_Z*2] = np.imag(kspace_und)
                                        kspace_und_re = np.float32(kspace_und_re)
                                        kspace_und_re = np.reshape(kspace_und_re,[1,dim_kspaceUnd_X,dim_kspaceUnd_Y,dim_kspaceUnd_Z*2])
                                        kspace_recon = kspace_und_re

                                        config = tf.ConfigProto()
                                        config.gpu_options.per_process_gpu_memory_fraction = 1/4 ; 

                                        for ind_c in range(0,no_channels):
                                            print('Reconstruting Channel #',ind_c+1)

                                            sess = tf.Session(config=config) 
                                            if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
                                                init = tf.initialize_all_variables()
                                            else:
                                                init = tf.global_variables_initializer()
                                            sess.run(init)

                                            # grab w and b
                                            w1 = np.float32(w1_all[:,:,:,:,ind_c])
                                            w2 = np.float32(w2_all[:,:,:,:,ind_c])     
                                            w3 = np.float32(w3_all[:,:,:,:,ind_c])

                                            if (b1_flag == 1):
                                                b1 = b1_all[:,:,:,ind_c];
                                            if (b2_flag == 1):
                                                b2 = b2_all[:,:,:,ind_c];
                                            if (b3_flag == 1):
                                                b3 = b3_all[:,:,:,ind_c];                

                                            res = cnn_3layer(kspace_und_re,w1,b1,w2,b2,w3,b3,acc_rate,sess) 
                                            target_x_end_kspace = dim_kspaceUnd_X - target_x_start;

                                            for ind_acc in range(0,acc_rate-1):

                                                target_y_start = np.int32((np.ceil(kernel_y_1/2)-1) + np.int32((np.ceil(kernel_y_2/2)-1)) + np.int32(np.ceil(kernel_last_y/2)-1)) * acc_rate + ind_acc + 1;             
                                                target_y_end_kspace = dim_kspaceUnd_Y - np.int32((np.floor(kernel_y_1/2)) + (np.floor(kernel_y_2/2)) + np.floor(kernel_last_y/2)) * acc_rate + ind_acc;
                                                kspace_recon[0,target_x_start:target_x_end_kspace,target_y_start:target_y_end_kspace+1:acc_rate,ind_c] = res[0,:,::acc_rate,ind_acc]

                                            sess.close()
                                            tf.reset_default_graph()

                                        kspace_recon = np.squeeze(kspace_recon)

                                        kspace_recon_complex = (kspace_recon[:,:,0:np.int32(no_channels/2)] + np.multiply(kspace_recon[:,:,np.int32(no_channels/2):no_channels],1j))
                                        kspace_recon_all_nocenter[:,:,:] = np.copy(kspace_recon_complex); 


                                        if no_ACS_flag == 0: 
                                            kspace_recon_complex[:,center_start:center_end,:] = kspace_NEVER_TOUCH[:,center_start:center_end,:]
                                            print('ACS signal has been putted back')
                                        else:
                                            print('No ACS signal is putted into k-space')

                                        kspace_recon_all[:,:,:] = kspace_recon_complex; 
                                        all_kspace_recons[:,:,:,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj] = kspace_recon_all

                                        time_ALL_end = time.time()
                                        print('All process costs ',(time_ALL_end-time_ALL_start)/60,'mins')
                                        print('Error Average in Training is ',errorSum/no_channels)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
R: 5 || acs: 20 || x1: 5 || y1: 2 || x2: 1 || y2: 3 || x3: 3 || y3: 2 || l1ch: 32 || l2ch: 8
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ACS signal found in the input data
go!
learning channel # 1
The 0 th iteration gives an error 0.028505582
The 100 th iteration gives an error 0.0017570087
The 200 th iteration gives an error 0.0012198179
The 300 th iteration gives an error 0.0009959892
The 400 th iteration gives an error 0.00095177034
The 500 th iteration gives an error 0.00083582365
The 600 th iteration gives an error 0.000770423
The 700 th iteration gives an error 0.00075960497
The 800 th iteration gives an error 0.00069919083
The 900 th iteration gives an error 0.0006950712
Time Cost: 19.808597803115845 s
Norm of Error =  0.0007074157
learning channel # 2
The 0 th iteration gives an error 0.02924171
The 100 th iteration gives

The 900 th iteration gives an error 0.00042775174
Time Cost: 2.6729836463928223 s
Norm of Error =  0.00041394294
learning channel # 15
The 0 th iteration gives an error 0.024549792
The 100 th iteration gives an error 0.001864806
The 200 th iteration gives an error 0.0012213954
The 300 th iteration gives an error 0.000991942
The 400 th iteration gives an error 0.0008802301
The 500 th iteration gives an error 0.0008453641
The 600 th iteration gives an error 0.0008042457
The 700 th iteration gives an error 0.00075382524
The 800 th iteration gives an error 0.0007357221
The 900 th iteration gives an error 0.0006687007
Time Cost: 2.748203992843628 s
Norm of Error =  0.00067124143
learning channel # 16
The 0 th iteration gives an error 0.022798775
The 100 th iteration gives an error 0.001474899
The 200 th iteration gives an error 0.0010161522
The 300 th iteration gives an error 0.0009217529
The 400 th iteration gives an error 0.0007803764
The 500 th iteration gives an error 0.0007310955
The 6

The 200 th iteration gives an error 0.00091166457
The 300 th iteration gives an error 0.0008131083
The 400 th iteration gives an error 0.0007695922
The 500 th iteration gives an error 0.0007256685
The 600 th iteration gives an error 0.0007138219
The 700 th iteration gives an error 0.0006643075
The 800 th iteration gives an error 0.00064951566
The 900 th iteration gives an error 0.0006485795
Time Cost: 2.7326457500457764 s
Norm of Error =  0.0006205641
learning channel # 30
The 0 th iteration gives an error 0.013697559
The 100 th iteration gives an error 0.001058507
The 200 th iteration gives an error 0.0007734886
The 300 th iteration gives an error 0.00067657523
The 400 th iteration gives an error 0.00061598344
The 500 th iteration gives an error 0.0005868999
The 600 th iteration gives an error 0.0005588495
The 700 th iteration gives an error 0.0005338747
The 800 th iteration gives an error 0.0005258915
The 900 th iteration gives an error 0.0005153013
Time Cost: 2.770860433578491 s
Nor

The 600 th iteration gives an error 0.0007903363
The 700 th iteration gives an error 0.0007309198
The 800 th iteration gives an error 0.00074169005
The 900 th iteration gives an error 0.00069744146
Time Cost: 2.773197889328003 s
Norm of Error =  0.000665408
learning channel # 44
The 0 th iteration gives an error 0.015729181
The 100 th iteration gives an error 0.0013978336
The 200 th iteration gives an error 0.00095122826
The 300 th iteration gives an error 0.0008243895
The 400 th iteration gives an error 0.00076424406
The 500 th iteration gives an error 0.0007354443
The 600 th iteration gives an error 0.00066363555
The 700 th iteration gives an error 0.00064472953
The 800 th iteration gives an error 0.00064257364
The 900 th iteration gives an error 0.0006051321
Time Cost: 2.764089822769165 s
Norm of Error =  0.00059294526
learning channel # 45
The 0 th iteration gives an error 0.008380124
The 100 th iteration gives an error 0.001053077
The 200 th iteration gives an error 0.0008694144
T

Time Cost: 2.7045915126800537 s
Norm of Error =  0.0008586464
learning channel # 58
The 0 th iteration gives an error 0.025438715
The 100 th iteration gives an error 0.0016664386
The 200 th iteration gives an error 0.0011326704
The 300 th iteration gives an error 0.00093165645
The 400 th iteration gives an error 0.0008604811
The 500 th iteration gives an error 0.0007896508
The 600 th iteration gives an error 0.0008039794
The 700 th iteration gives an error 0.00072723825
The 800 th iteration gives an error 0.0007166121
The 900 th iteration gives an error 0.00068847596
Time Cost: 2.691887378692627 s
Norm of Error =  0.00069178606
learning channel # 59
The 0 th iteration gives an error 0.038332295
The 100 th iteration gives an error 0.002881751
The 200 th iteration gives an error 0.001934646
The 300 th iteration gives an error 0.0014700847
The 400 th iteration gives an error 0.0013723054
The 500 th iteration gives an error 0.00124148
The 600 th iteration gives an error 0.0011676865
The 70

...

The 800 th iteration gives an error 0.00077788066
The 900 th iteration gives an error 0.00076303876
Time Cost: 2.6311113834381104 s
Norm of Error =  0.0007409875
learning channel # 36
The 0 th iteration gives an error 0.010564734
The 100 th iteration gives an error 0.0011407469
The 200 th iteration gives an error 0.00080324104
The 300 th iteration gives an error 0.00068734563
The 400 th iteration gives an error 0.0006176289
The 500 th iteration gives an error 0.000579019
The 600 th iteration gives an error 0.0005648324
The 700 th iteration gives an error 0.0005454468
The 800 th iteration gives an error 0.0005051098
The 900 th iteration gives an error 0.00051085645
Time Cost: 2.6300086975097656 s
Norm of Error =  0.00048449167
learning channel # 37
The 0 th iteration gives an error 0.019947331
The 100 th iteration gives an error 0.0021827829
The 200 th iteration gives an error 0.0014642652
The 300 th iteration gives an error 0.0012845049
The 400 th iteration gives an error 0.0011986558


The 100 th iteration gives an error 0.0016101145
The 200 th iteration gives an error 0.0011211253
The 300 th iteration gives an error 0.0009352356
The 400 th iteration gives an error 0.00085472135
The 500 th iteration gives an error 0.0007600971
The 600 th iteration gives an error 0.0007359669
The 700 th iteration gives an error 0.000686661
The 800 th iteration gives an error 0.0006604829
The 900 th iteration gives an error 0.00066973234
Time Cost: 2.7245070934295654 s
Norm of Error =  0.00062447606
learning channel # 51
The 0 th iteration gives an error 0.029263
The 100 th iteration gives an error 0.002446373
The 200 th iteration gives an error 0.0014338676
The 300 th iteration gives an error 0.0011595996
The 400 th iteration gives an error 0.0010740197
The 500 th iteration gives an error 0.000998531
The 600 th iteration gives an error 0.00095012283
The 700 th iteration gives an error 0.00089823426
The 800 th iteration gives an error 0.0008755578
The 900 th iteration gives an error 0.

The 600 th iteration gives an error 0.0011213772
The 700 th iteration gives an error 0.0010841186
The 800 th iteration gives an error 0.0010084672
The 900 th iteration gives an error 0.0009613172
Time Cost: 2.7474424839019775 s
Norm of Error =  0.0009125869
lerning step costs: 3.1897523482640584 min
Reconstruting Channel # 1
Reconstruting Channel # 2
Reconstruting Channel # 3
Reconstruting Channel # 4
Reconstruting Channel # 5
Reconstruting Channel # 6
Reconstruting Channel # 7
Reconstruting Channel # 8
Reconstruting Channel # 9
Reconstruting Channel # 10
Reconstruting Channel # 11
Reconstruting Channel # 12
Reconstruting Channel # 13
Reconstruting Channel # 14
Reconstruting Channel # 15
Reconstruting Channel # 16
Reconstruting Channel # 17
Reconstruting Channel # 18
Reconstruting Channel # 19
Reconstruting Channel # 20
Reconstruting Channel # 21
Reconstruting Channel # 22
Reconstruting Channel # 23
Reconstruting Channel # 24
Reconstruting Channel # 25
Reconstruting Channel # 26
Recons

The 100 th iteration gives an error 0.0025679502
The 200 th iteration gives an error 0.0018040736
The 300 th iteration gives an error 0.0015180707
The 400 th iteration gives an error 0.0013771651
The 500 th iteration gives an error 0.0012645602
The 600 th iteration gives an error 0.0011860055
The 700 th iteration gives an error 0.0011915703
The 800 th iteration gives an error 0.0011510084
The 900 th iteration gives an error 0.0011242267
Time Cost: 2.6466658115386963 s
Norm of Error =  0.0010904364
learning channel # 12
The 0 th iteration gives an error 0.012618954
The 100 th iteration gives an error 0.0019119954
The 200 th iteration gives an error 0.001438157
The 300 th iteration gives an error 0.0012499872
The 400 th iteration gives an error 0.0011341326
The 500 th iteration gives an error 0.0010789455
The 600 th iteration gives an error 0.0010073058
The 700 th iteration gives an error 0.0009811015
The 800 th iteration gives an error 0.00094903103
The 900 th iteration gives an error 0

The 600 th iteration gives an error 0.0016959283
The 700 th iteration gives an error 0.001620051
The 800 th iteration gives an error 0.0015390926
The 900 th iteration gives an error 0.0014659242
Time Cost: 3.085101842880249 s
Norm of Error =  0.0014200406
learning channel # 26
The 0 th iteration gives an error 0.022509824
The 100 th iteration gives an error 0.0020604476
The 200 th iteration gives an error 0.0014790805
The 300 th iteration gives an error 0.001207344
The 400 th iteration gives an error 0.0011157787
The 500 th iteration gives an error 0.0010353343
The 600 th iteration gives an error 0.001041929
The 700 th iteration gives an error 0.0009578042
The 800 th iteration gives an error 0.0009929278
The 900 th iteration gives an error 0.0008854051
Time Cost: 3.0668699741363525 s
Norm of Error =  0.0008497745
learning channel # 27
The 0 th iteration gives an error 0.063065074
The 100 th iteration gives an error 0.004887335
The 200 th iteration gives an error 0.0029144345
The 300 th

#  computing all reconstructions a rmses

In [ ]:
ablation_dims = [len(all_kernel_x_1),len(all_kernel_y_1),len(all_kernel_x_2),len(all_kernel_y_2),\
                len(all_kernel_last_x),len(all_kernel_last_y),len(all_layer1_channels),len(all_layer2_channels),\
                len(all_Ry),len(all_acsy)]
all_rmse   = np.zeros(ablation_dims)

all_recons = sig.rsos(ifft2c_raki(all_kspace_recons),2)
truth      = sig.rsos(ifft2c_raki(kspace_truth_raki),2)

for aa in range(len(all_kernel_x_1)):
    for bb in range(len(all_kernel_y_1)):
        for cc in range(len(all_kernel_x_2)):
            for dd in range(len(all_kernel_y_2)):
                for ee in range(len(all_kernel_last_x)):
                    for ff in range(len(all_kernel_last_y)):
                        for gg in range(len(all_layer1_channels)):
                            for hh in range(len(all_layer2_channels)):
                                for ii in range(len(all_Ry)):
                                    for jj in range(len(all_acsy)):
                                        all_rmse[aa,bb,cc,dd,ee,ff,gg,hh,ii,jj] = \
                                            sig.rmse(sig.nor(truth),sig.nor(all_recons[:,:,aa,bb,cc,dd,ee,ff,gg,hh,ii,jj]))

## Find parameter set with minimum rmse and then display 

In [ ]:
cur_rmse = 1e16

for aa in range(len(all_kernel_x_1)):
    for bb in range(len(all_kernel_y_1)):
        for cc in range(len(all_kernel_x_2)):
            for dd in range(len(all_kernel_y_2)):
                for ee in range(len(all_kernel_last_x)):
                    for ff in range(len(all_kernel_last_y)):
                        for gg in range(len(all_layer1_channels)):
                            for hh in range(len(all_layer2_channels)):
                                for ii in range(len(all_Ry)):
                                    for jj in range(len(all_acsy)):
                                        if(all_rmse[aa,bb,cc,dd,ee,ff,gg,hh,ii,jj] < cur_rmse):
                                            best_params = [aa,bb,cc,dd,ee,ff,gg,hh,ii,jj]
                                            cur_rmse = all_rmse[aa,bb,cc,dd,ee,ff,gg,hh,ii,jj]

print("Best Params:")
print("R: %d || acs: %d || x1: %d || y1: %d || x2: %d || y2: %d || x3: %d || y3: %d || l1ch: %d || l2ch: %d" % \
      (all_Ry[best_params[8]],all_acsy[best_params[9]],all_kernel_x_1[best_params[0]],\
       all_kernel_y_1[best_params[1]],all_kernel_x_2[best_params[2]],all_kernel_y_2[best_params[3]], \
       all_kernel_last_x[best_params[4]],all_kernel_last_y[best_params[5]],\
       all_layer1_channels[best_params[6]],all_layer2_channels[best_params[7]]))
print("Best rmse: %.2f" %(cur_rmse * 100))

# Quick display 

In [ ]:
raki = np.expand_dims(np.transpose(all_recons[:,:,best_params[0],best_params[1],best_params[2],best_params[3],best_params[4],\
                              best_params[5],best_params[6],best_params[7],best_params[8],best_params[9]],(1,0)),axis = 0)

truth   = np.expand_dims(np.transpose(sig.rsos(ifft2c_raki(kspace_truth_raki),-1),(1,0)),axis=0)
display = np.concatenate((sig.nor(truth),sig.nor(raki)),axis = 0)
sig.mosaic(display,1,2)

# Assuming I'm only doing ablation over acs size and acceleration, save 

In [ ]:
results = {'truth':   np.squeeze(truth),
           'all_raki' :   np.squeeze(all_recons),
           'all_raki_rmse': np.squeeze(all_rmse),
           'accelerations': all_Ry,
           'acs_sizes': all_acsy}

sio.savemat('results/raki_ablation.mat', results, oned_as='row')